# 라이브러리 불러오기

- -q를 사용해도 깔끔하게 라이브러리를 불러올 수 있지만, %%capture를 사용해도 깔끔하게 라이브러리를 불러올 수 있습니다.  

In [ ]:
%%capture
!pip install transformers bitsandbytes datasets sentencepiece accelerate trl peft flash-attn wandb openai pqdm

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

import warnings
warnings.filterwarnings("ignore")

import trl
import torch
import datasets
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, AutoPeftModelForCausalLM

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)

from huggingface_hub import login

import os
import json
from openai import OpenAI

In [ ]:
print(f"PyTorch version       : {torch.__version__}")
print(f"Transformers version  : {transformers.__version__}")
print(f"TRL version           : {trl.__version__}")
print(f"CUDA available        : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version      : {torch.version.cuda}")

PyTorch version       : 2.3.1+cu121
Transformers version  : 4.42.4
TRL version           : 0.9.6
CUDA available        : True
CUDA version      : 12.1


In [ ]:
login(
  token="Your_Huggingface_API_KEY", # 여기에 토큰 추가 
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


이 코드는 Hugging Face API 토큰을 사용하여 Hugging Face Hub에 인증하는 코드이며, 이를 통해 Hugging Face의 모델, 데이터셋, 기타 리소스에 접근할 수 있습니다.

token="Your Token"

Your Token 자리에 huggingface에서 발급받은 코드를 넣습니다.

In [ ]:
dataset = datasets.load_dataset("daje/kotext-to-sql-v1")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'ko_instruction'],
        num_rows: 262208
    })
})

텍스트 - SQL 로 연결된 데이터 입니다.

In [ ]:
def add_length_column(dataset):
    df = dataset.to_pandas()
    df["total_length"] = 0
    for column_name in ["ko_instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

df = add_length_column(dataset["train"])

def filter_by_total_length(df, difficulty, number_of_samples):
    if difficulty == "easy":
        return df[df["total_length"].between(10, 100)].iloc[:number_of_samples]
    elif difficulty == "moderate":
        return df[df["total_length"].between(101, 300)].iloc[:number_of_samples]
    elif difficulty == "difficult":
        return df[df["total_length"].between(301, 1000)].iloc[:number_of_samples]

print(max(df["total_length"].to_list()), min(df["total_length"].to_list()))

910 13


In [ ]:
df.head(2)

,instruction,input,response,source,text,ko_instruction,total_length
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...,sql_create_context,Below are sql tables schemas paired with instr...,카를턴의 원정 팀의 홈 팀 이름을 말해 주세요.,25
1,what will the population of Asia be when Latin...,"CREATE TABLE table_22767 (\n ""Year"" real,\n...","SELECT ""Asia"" FROM table_22767 WHERE ""Latin Am...",wikisql,Below are sql tables schemas paired with instr...,"라틴 아메리카/카리브해 지역의 인구가 783(7.5%)가 될 때, 아시아의 인구는 ...",44


In [ ]:
easy = filter_by_total_length(df, "easy", 5000)
medium = filter_by_total_length(df, "moderate", 5000)
hard = filter_by_total_length(df, "difficult", 5000)

dataset = pd.concat([easy, medium, hard])

dataset = dataset.sample(frac=1)
dataset = Dataset.from_pandas(dataset)
easy.shape, medium.shape, hard.shape, dataset.shape

((5000, 6), (5000, 6), (5000, 6), (15000, 6))

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = "You are a helpful programmer assistant that excels at SQL."
    user_prompt = "Task: {ko_instruction}\nSQL table: {input}\nSQL query: "
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},
        ]
    }

# 데이터 전처리를 위해 먼저 도크나이저를 불러옵니다. 
# 자세한 설명은 ###4.2.4 섹션에서 설명합니다. 
tokenizer = AutoTokenizer.from_pretrained("allganize/Llama-3-Alpha-Ko-8B-Instruct")   
# 시퀀스의 길이를 맞추기 위해 추가되는 특별한 토큰으로, 일반적으로 pad_token_id에 해당합니다. 
# 패딩 방향을 설정함으로써 모델이 일관된 입력 형식을 받을 수 있도록 합니다.
tokenizer.padding_side = 'right'                      

# 데이터를 일괄적으로 대화형식으로 변경합니다.
dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)

# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset = dataset.train_test_split(test_size=0.05)

# json으로 저장합니다.
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

# 정상적으로 변환되었는지 확인합니다.
print(dataset["train"][345]["messages"])


Map:   0%|          | 0/262208 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/250 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

[{'content': 'You are a helpful programmer assistant that excels at SQL.', 'role': 'system'}, {'content': 'Task: 츠구이 마츠다가 운전하는 모든 라운드에서 어떤 엔진이 사용되나요?\nSQL table: CREATE TABLE table_48913 (\n    "Team" text,\n    "Driver" text,\n    "Chassis" text,\n    "Engine" text,\n    "Rounds" text\n)\nSQL query: ', 'role': 'user'}, {'content': 'SELECT "Engine" FROM table_48913 WHERE "Rounds" = \'all\' AND "Driver" = \'tsugio matsuda\'<|im_end|>', 'role': 'assistant'}]


In [ ]:
# 정성적으로 변환되었는지 확인합니다.
dataset["train"], dataset["test"]

(Dataset({
     features: ['messages'],
     num_rows: 249097
 }),
 Dataset({
     features: ['messages'],
     num_rows: 13111
 }))

In [ ]:
# 데이터가 정상적으로 채팅 포맷으로 변경되었는지 확인합니다.
dataset["train"][0]

{'messages': [{'content': 'You are a helpful programmer assistant that excels at SQL.',
   'role': 'system'},
  {'content': 'Task: 장관들과 그들이 취임하고 퇴임한 시간을 퇴임 시간을 기준으로 나열해서 보여주세요.\nSQL table: CREATE TABLE party_events (\n    event_id number,\n    event_name text,\n    party_id number,\n    member_in_charge_id number\n)\n\nCREATE TABLE member (\n    member_id number,\n    member_name text,\n    party_id text,\n    in_office text\n)\n\nCREATE TABLE party (\n    party_id number,\n    minister text,\n    took_office text,\n    left_office text,\n    region_id number,\n    party_name text\n)\n\nCREATE TABLE region (\n    region_id number,\n    region_name text,\n    date text,\n    label text,\n    format text,\n    catalogue text\n)\nSQL query: ',
   'role': 'user'},
  {'content': 'SELECT minister, took_office, left_office FROM party ORDER BY left_office<|im_end|>',
   'role': 'assistant'}]}

In [ ]:
# 저장된 train 데이터를 불러옵니다.
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Quantization config 세팅 -> 모델이 사용하는 vram을 최소화하기
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # double quantization으로 양자화 오류를 줄입니다.
    bnb_4bit_use_double_quant=True, 
    # 다양한 양자화 종류 중 nf4를 선택
    bnb_4bit_quant_type="nf4", 
    # llama는 16비트 부동 소수를 사용해 학습됐습니다.
    bnb_4bit_compute_dtype=torch.bfloat16 
)

# 이번 프로젝트에서 사용할 모델로 LLama2를 Base Model로 사용하여 코드를 전용으로 만든 모델
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

# 모델과 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained(
    # 앞서 정의한 모델을 불러옵니다.
    model_id,                                     
    # 모델을 사용할 디바이스를 자동으로 설정합니다.
    device_map="auto",                            
    # 더 빠르고 메모리 효율적인 어텐션 구현 방식입니다.
    attn_implementation="flash_attention_2",       
    torch_dtype=torch.bfloat16,
    # 양자화 설정을 적용합니다.
    quantization_config=bnb_config                  
)
# 토크나이저 불러옵니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)   
# 시퀀스의 길이를 맞추기 위해 추가되는 특별한 토큰으로, 일반적으로 pad_token_id에 해당합니다. 
# 패딩 방향을 설정함으로써 모델이 일관된 입력 형식을 받을 수 있도록 합니다.
tokenizer.padding_side = 'right'                      

# setup_chat_format 함수를 사용하는 주요 이유는 모델과 토크나이저가 대화형 AI 시스템에서 요구하는 형식에 맞게 추가 설정을 적용하기 위함입니다. 
# 이 함수는 특별 토큰 추가, 입력 형식 포맷팅, 토큰 임베딩 조정 등의 작업을 수행하여 모델이 대화형 응답을 보다 효과적으로 생성할 수 있도록 준비합니다.
model, tokenizer = setup_chat_format(model, tokenizer) 

In [ ]:
peft_config = LoraConfig(
        lora_alpha=128,                            
        lora_dropout=0.05,                         # Lora 학습 때 사용할 dropout 확률을 지정합니다. 드롭아웃 확률은 과적합 방지를 위해 학습 중 무작위로 일부 뉴런을 비활성화하는 비율을 지정합니다.
        r=256,                                     # Lora의 저차원 공간의 랭크를 지정합니다. 랭크가 높을수록 모델의 표현력이 증가하지만, 계산 비용도 증가합니다.
        bias="none",                               # Lora 적용 시 바이어스를 사용할지 여부를 설정합니다. "none"으로 설정하면 바이어스를 사용하지 않습니다.
        target_modules=["q_proj", "o_proj",        # Lora를 적용할 모델의 모듈 리스트입니다.
                        "k_proj", "v_proj"
                        "up_proj", "down_proj",
                        "gate_proj",
                        ],
        task_type="CAUSAL_LM",                     # 미세 조정 작업 유형을 CAUSAL_LM으로 지정하여 언어 모델링 작업을 수행합니다.
)


args = TrainingArguments(
    output_dir="code-llama-7b-text-to-sql", # 모델 저장 및 허브 업로드를 위한 디렉토리 지정 합니다.
    num_train_epochs=1,                   # number of training epochs
    # max_steps=100,                          # 100스텝 동안 훈련 수행합니다.
    per_device_train_batch_size=1,          # 배치 사이즈 설정 합니다.
    gradient_accumulation_steps=2,          # 4스텝마다 역전파 및 가중치 업데이트합니다.
    gradient_checkpointing=True,            # 메모리 절약을 위해 그래디언트 체크포인팅 사용합니다.
    optim="adamw_torch_fused",              # 메모리 효율화할 수 있는 fused AdamW 옵티마이저 사용합니다.
    logging_steps=10,                       # 10스텝마다 로그 기록합니다.
    save_strategy="epoch",                  # 매 에폭마다 체크포인트 저장합니다.
    learning_rate=2e-4,                     # 학습률 2e-4로 설정 (QLoRA 논문 기반)합니다.
    bf16=True,                              # 정밀도 설정으로 학습 속도 향상합니다.
    tf32=True,
    max_grad_norm=0.3,                      # 그래디언트 클리핑 값 0.3으로 설정합니다.
    warmup_ratio=0.03,                      # 워밍업 비율 0.03으로 설정 (QLoRA 논문 기반)합니다.
    lr_scheduler_type="constant",           # 일정한 학습률 스케줄러 사용합니다.
    push_to_hub=True,                       # 훈련된 모델을 Hugging Face Hub에 업로드합니다.
    report_to="wandb",                      # wandb로 매트릭 관찰합니다.
)


max_seq_length = 7994 # 최대 시퀀스 길이 설정

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,                     # 패킹은 여러 시퀀스를 하나의 고정된 크기 내에서 묶어 처리하는 방법입니다. 이를 통해 효율적인 배치 처리가 가능합니다.
    dataset_kwargs={
        "add_special_tokens": False,  # 템플릿에 특별 토큰을 추가하지 않습니다. 이는 이미 템플릿에 특별 토큰이 포함되어 있음을 의미할 수 있습니다.
        "append_concat_token": False, # 추가 구분자 토큰을 추가하지 않습니다. 데이터셋의 항목들이 이미 적절히 구분되어 있음을 의미합니다.
    }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the o

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# trainer를 학습합니다.
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: daje0601. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,0.918700
20,0.548900
30,0.435300
40,0.384900
50,0.424000
60,0.617600
70,0.373300
80,0.336600
90,0.366400
100,0.363200


Step,Training Loss
10,0.918700
20,0.548900
30,0.435300
40,0.384900
50,0.424000
60,0.617600
70,0.373300
80,0.336600
90,0.366400
100,0.363200


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=4539, training_loss=0.3145092622276234, metrics={'train_runtime': 35155.5457, 'train_samples_per_second': 0.258, 'train_steps_per_second': 0.129, 'total_flos': 3.475971809779876e+18, 'train_loss': 0.3145092622276234, 'epoch': 0.9998898557109814})

In [ ]:
model.push_to_hub("daje/code-llama3-8B-text-to-sql-ver0.1")
tokenizer.push_to_hub("daje/code-llama3-8B-text-to-sql-ver0.1")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/daje/code-llama3-8B-text-to-sql-ver0.1/commit/4bcb670f7d0a6ef6308c0dc076dee7a54823c81a', commit_message='Upload tokenizer', commit_description='', oid='4bcb670f7d0a6ef6308c0dc076dee7a54823c81a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# 메모리 초기화
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
# 학습한 모델을 경로를 지정합니다.
peft_model_id = "./code-llama3-8B-text-to-sql"

# PEFT 어댑터를 통해 사전 학습된 모델을 로드합니다.
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.bfloat16
)

# 토크나이저 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# 생성을 조금 더 효율적으로 하기 위해 파이프라인을 불러옵니다.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCau

In [ ]:
# 테스트 데이터를 불러옵니다.
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# 샘플 데이터 설정합니다.
prompt = pipe.tokenizer.apply_chat_template(
    eval_dataset[rand_idx]["messages"][:2], 
    tokenize=False, 
    add_generation_prompt=True
    )

outputs = pipe(prompt, 
               max_new_tokens=256, 
               do_sample=False, 
               temperature=0.1, 
               top_k=50, 
               top_p=0.1, 
               eos_token_id=pipe.tokenizer.eos_token_id, 
               pad_token_id=pipe.tokenizer.pad_token_id
               )

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}".replace("<|im_end|>", ""))
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
eval_dataset[rand_idx]['messages'][2]['content'].replace("<|im_end|>", "") == outputs[0]['generated_text'][len(prompt):].strip()

Generating train split: 0 examples [00:00, ? examples/s]

Query:
Task: '말이 spender s 인 경우의 최고 총액을 말해줘.'
SQL table: CREATE TABLE table_12014 (
    "Rider" text,
    "Horse" text,
    "Faults" text,
    "Round 1 + 2A Points" text,
    "Total" real
)
SQL query: 
Original Answer:
SELECT MAX("Total") FROM table_12014 WHERE "Horse" = 'spender s'
Generated Answer:
SELECT MAX("Total") FROM table_12014 WHERE "Horse" ='spender s'


False

In [ ]:
from tqdm import tqdm

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(
        sample["messages"][:2],
        tokenize=False,
        add_generation_prompt=True)
    outputs = pipe(prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=pipe.tokenizer.eos_token_id,
        pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    return (sample["messages"][1]["content"], predicted_answer, sample["messages"][2]["content"])

success_rate = []
number_of_eval_samples = 1500

sampled_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1500))
for test_data in tqdm(sampled_eval_dataset):
    success_rate.append(evaluate(test_data))

In [ ]:
success_rate[0][1]

"SELECT COUNT(*) FROM lab WHERE lab.patientunitstayid IN (SELECT patient.patientunitstayid FROM patient WHERE patient.patienthealthsystemstayid IN (SELECT patient.patienthealthsystemstayid FROM patient WHERE patient.uniquepid = '004-86136')) AND lab.labname ='mchc' AND DATETIME(lab.labresulttime) >= DATETIME(CURRENT_TIME(), '-80 month')"

In [ ]:
success_rate[0][2].replace("<|im_end|>", "")

"SELECT COUNT(*) FROM lab WHERE lab.patientunitstayid IN (SELECT patient.patientunitstayid FROM patient WHERE patient.patienthealthsystemstayid IN (SELECT patient.patienthealthsystemstayid FROM patient WHERE patient.uniquepid = '004-86136')) AND lab.labname = 'mchc' AND DATETIME(lab.labresulttime) >= DATETIME(CURRENT_TIME(), '-80 month')"

In [ ]:
success_rate[0][1].strip() == success_rate[0][2].replace("<|im_end|>", "").strip()

False

In [ ]:
with open("/content/drive/MyDrive/success_rate.txt", "w") as f:
    for result in success_rate:
        f.write(str(result) + "\n")

In [ ]:
# 데이터를 살펴보겠습니다.
print(success_rate[1][0])
print("-" * 10)
print(success_rate[1][1])

SELECT "Club" FROM table_50722 WHERE "Runners-up" = '1' AND "Winners" = '0' AND "Third" = '2'
----------
SELECT "Club" FROM table_50722 WHERE "Runners-up" = '1' AND "Winners" = '0' AND "Third" = '2'<|im_end|>


In [ ]:
# 그러나, 이런 생성결과는 ACC로만 보기에는 무리가 있습니다.
print(success_rate[20][0])
print("-" * 10)
print(success_rate[20][1])

SELECT H.AnswerDate, COUNT(*) FROM (SELECT Id AS AnswerId, CreationDate AS AnswerDate FROM Posts WHERE PostTypeId = 2) AS H GROUP BY H.AnswerDate ORDER BY COUNT(*) DESC
----------
SELECT TIME_TO_STR(p.CreationDate, '%h'), COUNT(*) FROM Posts AS p WHERE p.OwnerUserId = @UserId AND p.PostTypeId = 2 GROUP BY TIME_TO_STR(p.CreationDate, '%h')<|im_end|>


In [ ]:
# 이 데이터 또한 exact match로는 False가 납니다.
print(success_rate[350][0])
print("-" * 10)
print(success_rate[350][1])

SELECT date_test_taken FROM student_tests_taken WHERE test_result = 'Pass'
----------
SELECT date_test_taken FROM student_tests_taken WHERE test_result = "Pass"<|im_end|>


In [ ]:
# eos_token을 일괄적으로 제거합니다.
generated_result = [temp[1] == temp[2].replace("<|im_end|>", "") for temp in success_rate]

In [ ]:
# Exact Match 기준으로 ACC(정확도)를 구합니다.
accuracy = sum(generated_result)/len(generated_result)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 56.87%


In [ ]:
success_rate = []
with open("/content/drive/MyDrive/success_rate_20240728.txt", "r") as f:
    for line in f:
        success_rate.append(eval(line))

### 4.2.11 OpenAI API로 평가하기 

In [ ]:
openai_evaluation = [(temp[0], temp[1], temp[2].replace("<|im_end|>", "")) for temp in success_rate]

In [ ]:
openai_evaluation[1]

('Task: tf103 차대의 총 포인트 수를 알려주세요.\nSQL table: CREATE TABLE table_11482 (\n    "Year" real,\n    "Chassis" text,\n    "Engine" text,\n    "Tyres" text,\n    "Points" real\n)\nSQL query: ',
 'SELECT COUNT("Points") FROM table_11482 WHERE "Chassis" = \'tf103\'',
 'SELECT COUNT("Points") FROM table_11482 WHERE "Chassis" = \'tf103\'')

In [ ]:
# gpt-4o-mini를 사용해서 문제와 정답과 생성된 결과를 넣고, 같은 쿼리인지 확인
# OpenAI API 키 설정 (환경 변수에서 가져오거나 직접 입력)
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_API_KEY"

client = OpenAI()

def one_compare_sql_semantics(problem_description, generated_query, ground_truth_query):
    # ChatGPT에게 물어볼 프롬프트 작성
    prompt = f"""다음 문제와 두 SQL 쿼리가 의미적으로 동일한 결과를 반환하는지 판단해주세요:

    문제 설명: {problem_description}

    생성된 쿼리:
    {generated_query}

    정답 쿼리:
    {ground_truth_query}

    두 쿼리가 문제에 대해 의미적으로 동일한 결과를 반환한다면 "Yes"라고 대답하고,
    그렇지 않다면 "No"라고 대답한 후 차이점을 설명해주세요.
    쿼리의 구조나 사용된 함수가 다르더라도 결과가 같다면 의미적으로 동일하다고 판단해주세요."""

    # ChatGPT API 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 또는 사용 가능한 최신 모델
        messages=[
            {"role": "system", "content": "You are a helpful assistant that compares the semantic meaning of SQL queries in the context of a given problem."},
            {"role": "user", "content": prompt}
        ]
    )

    # ChatGPT의 응답 추출
    answer = response.choices[0].message.content.strip()

    # 결과 처리
    is_correct = 1 if answer.lower().startswith("yes") else 0
    explanation = answer[3:] if is_correct == 1 else answer[2:]

    # JSON 형식으로 결과 반환
    result = {
        "answer": is_correct,
        "explanation": explanation.strip()
    }

    return json.dumps(result, ensure_ascii=False)

# 사용 예시

problem = openai_evaluation[1][0]
truth = openai_evaluation[1][1]
generated = openai_evaluation[1][2]

result = one_compare_sql_semantics(problem, generated, truth)
print(result)

{"answer": 0, "explanation": "es\"\n\n두 쿼리는 완전히 동일합니다. 모두 \"table_11482\" 테이블에서 \"Chassis\"가 'tf103'인 행을 필터링하고, 그 행의 \"Points\" 컬럼의 개수를 계산하는 COUNT 함수가 사용되었습니다. 결과적으로 두 쿼리는 차대(tf103)에 대한 총 포인트 수를 반환하므로 의미적으로 동일한 결과를 제공합니다."}


In [ ]:
import os
import json
from pathlib import Path
from openai import OpenAI
from pqdm.processes import pqdm

# OpenAI API 키 설정 (환경 변수에서 가져오거나 직접 입력)
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_API_KEY"

client = OpenAI()

def compare_sql_semantics(idx):
    save_path = f"/content/drive/MyDrive/text_to_sql_result_ver0.1/result_{idx}.json"
    if Path(save_path).exists():
        print("이미 처리된 파일입니다.")
        pass
    else:
        item = generated_result[idx]
        problem_description, generated_query, ground_truth_query = item

        # ChatGPT에게 물어볼 프롬프트 작성
        prompt = f"""다음 문제와 두 SQL 쿼리가 의미적으로 동일한 결과를 반환하는지 판단해주세요:

        문제 설명: {problem_description}

        생성된 쿼리:
        {generated_query}

        정답 쿼리:
        {ground_truth_query}

        두 쿼리가 문제에 대해 의미적으로 동일한 결과를 반환한다면 answer에 "1"라고 대답하고,
        그렇지 않다면 "0"라고 대답한 후 차이점을 explanation에 적으세요.
        쿼리의 구조나 사용된 함수가 다르더라도 결과가 같다면 의미적으로 동일하다고 판단해주세요."""

        # ChatGPT API 호출
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 또는 사용 가능한 최신 모델
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": """You are a helpful assistant that compares the semantic meaning of SQL queries in the context of a given problem.
                return json format below:
                {
                    "answer": "...",
                    "explanation": "..."
                }
                """},
                {"role": "user", "content": prompt}
            ]
        )

        # ChatGPT의 응답 추출
        answer = response.choices[0].message.content.strip()

        # 결과를 JSON 파일로 저장
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(answer, f, ensure_ascii=False, indent=4)

        return answer

# generated_result에 인덱스 추가
indexed_openai_evaluation = list(range(len((openai_evaluation))))

# pqdm을 사용하여 병렬 처리
results = pqdm(indexed_openai_evaluation, compare_sql_semantics, n_jobs=40)

QUEUEING TASKS | :   0%|          | 0/1500 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1500 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1500 [00:00<?, ?it/s]

1500개 평가하는데 약 20분 정도 소요됩니다.

In [ ]:
results[:3]

['{\n    "answer": "0",\n    "explanation": "첫 번째 쿼리에서 \'try_bonus\' 값이 정수 140으로 비교되고, 두 번째 쿼리에서는 문자열 \'140\'으로 비교됩니다. 이로 인해 두 쿼리는 타입이 서로 다르기 때문에 의미적으로 동일한 결과를 반환하지 않습니다."\n}',
 '{\n    "answer": "1",\n    "explanation": "두 쿼리는 동일하며, 같은 조건을 사용하여 동일한 결과를 반환합니다. 따라서 의미적으로 동일한 결과를 반환한다고 판단됩니다."\n}',
 '{\n    "answer": "1",\n    "explanation": "두 쿼리는 구조적으로 동일하며, \'Aggregate\'가 \'4-2\'인 경우에 대해 \'2nd leg\' 값을 선택하므로 의미적으로 동일한 결과를 반환합니다."\n}']

In [ ]:
json_result = []
for result in results:
    json_result.append(json.loads(result))

df = pd.DataFrame(json_result)

df["answer"] = df["answer"].map(lambda x : int(x))

after_accuracy = df["answer"].sum() / len(df["answer"])
print(f"Accuracy: {after_accuracy*100:.2f}%")

Accuracy: 62.80%
